In [2]:
import os
import warnings
import os
from bertopic import BERTopic
import subprocess
import time
import requests
import json
import pandas as pd
import ollama
from pydantic import BaseModel, Field, ValidationError
from typing import List, Literal, Optional
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import kagglehub
import bz2
from transformers import pipeline
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

warnings.filterwarnings("ignore")
os.environ["OMP_NUM_THREADS"] = "1"
warnings.filterwarnings("ignore")

In [3]:

# Configuration
INPUT_JSON_PATH = "dk_posts_synth_en_processed.json"   # Path to a JSON file containing a *list* of dictionaries
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Model Parameters
EMBEDDING_MODEL_NAME = "intfloat/multilingual-e5-small"
MIN_TOPIC_SIZE = 10       # Minimum number of documents to form a topic
N_NEIGHBORS = 12          # Used by UMAP for dimensionality reduction
N_COMPONENTS = 5          # Used by UMAP for dimensionality reduction
RANDOM_STATE = 42         # Ensures reproducible results

In [2]:
import kagglehub

# Download latest version
df=path = kagglehub.dataset_download("bittlingmayer/amazonreviews")

print("Path to dataset files:", df)


Path to dataset files: /Users/christianorynielsen/.cache/kagglehub/datasets/bittlingmayer/amazonreviews/versions/7


In [3]:
import requests, json

url = "http://localhost:11434/api/generate"
data = {
    "model": "gemma3:12b",
    "prompt": "Say hello from Ollama!"
}

response = requests.post(url, json=data, stream=True)

full_response = ""
for line in response.iter_lines():
    if line:
        content = json.loads(line.decode("utf-8"))
        if "response" in content:
            full_response += content["response"]

print(full_response)

Hello from Ollama! 👋 



It's nice to connect! 😊


In [5]:
# Correctly open and read the BZ2-compressed text file
file_path = f"{path}/train.ft.txt.bz2"

data = []
with bz2.open(file_path, "rt", encoding="utf-8", errors="ignore") as f:
    for i, line in enumerate(f):
        if i >= 1000:  # keep it small for now
            break
        parts = line.strip().split(" ", 1)
        if len(parts) != 2:
            continue
        label, text = parts
        data.append((label, text))

# Create DataFrame
df = pd.DataFrame(data, columns=["label", "review"])

# Clean label names
df["label"] = df["label"].replace({"__label__1": "negative", "__label__2": "positive"})

# Preview
print(df["label"].value_counts())
df.head()



label
negative    538
positive    462
Name: count, dtype: int64


,label,review
0,positive,Stuning even for the non-gamer: This sound tra...
1,positive,The best soundtrack ever to anything.: I'm rea...
2,positive,Amazing!: This soundtrack is my favorite music...
3,positive,Excellent Soundtrack: I truly like this soundt...
4,positive,"Remember, Pull Your Jaw Off The Floor After He..."
